# Python decorator
A tutorial link: https://realpython.com/primer-on-python-decorators/#functions

A Function is defined as follows

In [12]:
# Function 1
def func():
    def func1():
        print('This is an inside function')
        flag =False
        return flag
    func1()
    if True:
        print('This is a function')
        flag = True
        return flag
    
# Calling the function
print(func())
func1() # scope error

This is an inside function
This is a function
True


NameError: name 'func1' is not defined

# Simple, old-fashioned decorators
Decorators wrap a function, modifying its behavior. In the following example, we send in the `say_whee` function to `my_decorator` and returns `wrapper` which is reassigned to `say_whee`.

In [17]:
def my_decorator(func):
    def wrapper():
        print("Something is happening before the function is called.")
        func() # this is say_shee function
        print("Something is happening after the function is called.")
    return wrapper

def say_whee():
    print("Whee!")

say_whee = my_decorator(say_whee)
print('say_whee is pointing to : ',say_whee)
say_whee()

say_whee is pointing to :  <function my_decorator.<locals>.wrapper at 0x7fdd3e980940>
Something is happening before the function is called.
Whee!
Something is happening after the function is called.


Here is another example: this code will only work when `datetime.now().hour` is between $\left[5,22\right]$.

In [25]:
from datetime import datetime

def not_during_the_night(func):
    def wrapper():
        if 5 <= datetime.now().hour < 22:
            func()
        else:
            pass  # Hush, the neighbors are asleep
    return wrapper

def say_whee():
    print("Whee!")

say_whee = not_during_the_night(say_whee)

In [26]:
datetime.now().hour

6

In [27]:
say_whee()

Whee!


# Standard decorators
Python allows you to use decorators in a simpler way with the `@` symbol, sometimes called the “pie” syntax. So, `@my_decorator` is just an easier way of saying `say_whee = my_decorator(say_whee)`. It’s how you apply a decorator to a function.

In [ ]:
def my_decorator(func):
    def wrapper():
        print("Something is happening before the function is called.")
        func()
        print("Something is happening after the function is called.")
    return wrapper

@my_decorator
def say_whee():
    print("Whee!")
print(say_whee)   # function pinter
print(say_whee()) # calling the function

<function my_decorator.<locals>.wrapper at 0x7fdd3e4139d0>
Something is happening before the function is called.
Whee!
Something is happening after the function is called.
None


# reusing decorators
Recall that a decorator is just a regular Python function. All the usual tools for easy reusability are available. Let’s move the decorator to its own module that can be used in many other functions. Create a file called `decorators.py` with the following content:

In [29]:
def do_twice(func):
    def wrapper():
        func()
        func()
    return wrapper

In [2]:
# check decorator.py file
from decorator_1 import do

@do
def func():
    print('hello')
    
func, func()

hello
hello


(<function decorator_1.do.<locals>.wrapper()>, None)

# Decorating Functions With Arguments
use `*args` and `**kwargs` in the inner wrapper function. Then it will accept an arbitrary number of positional and keyword arguments. Rewrite `decorators.py` as follows:
```python
def do_twice(func):
    def wrapper(*args, **kwargs):
        func(*args, **kwargs)
        func(*args, **kwargs)
    return wrapper
```

In [5]:
# check decorator.py file
from decorator_1 import do_twice

@do_twice
def func_1(*args, **kwargs):
    print(str(*args))
    
func_1, func_1('name')

name
name


(<function decorator_1.do_twice.<locals>.wrapper_1(*args, **kwargs)>, None)

# Returning Values From Decorated Functions
So far we have returned functions using a `decorator`. Now we will use decorated function to return values. We have to add a return value to our wrapper in `decorator.py`.

In [3]:
# check decorator.py file
from decorator_1 import do_twice

@do_twice
def func_1(name):
    print(name)
    return f"hi {name}" 
    
func_1, func_1('name')

name1
name1
name1


(<function decorator_1.do_twice.<locals>.wrapper_1(*args, **kwargs)>,
 'hi name')

In [2]:
a=func_1('name')
a

name
name
name


'hi name'

In [6]:
# Every function knows its name
func_1.__name__, do_twice.__name__

('wrapper_1', 'do_twice')

In [13]:
help(func_1),print('\n\n'), help(do_twice)

Help on function wrapper_1 in module decorator_1:

wrapper_1(*args, **kwargs)




Help on function do_twice in module decorator_1:

do_twice(func)



(None, None, None)

However, after being decorated, `func_1` has gotten very confused about its identity. It now reports being the `wrapper_1()` inner function inside the `do_twice()` decorator. Although technically true, this is not very useful information. To fix this, decorators should use the `@functools.wraps` decorator, which will preserve information about the original function. Update `decorators.py` again:

```python
import functools

def do_twice(func):
    @functools.wraps(func)
    def wrapper_1(*args, **kwargs):
        func(*args, **kwargs)
        func(*args, **kwargs)
        return func(*args, **kwargs)
    return wrapper_1
```

In [2]:
from decorator_1 import do_twice_functools

@do_twice_functools
def func_1():
    print('hello')
    
func_1,func_1()

hello
hello
hello


(<function __main__.func_1()>, None)

In [4]:
# This time the identity should be the same
func_1.__name__

'func_1'

# A Decorator boilerplate  template
```python
import functools

def decorator(func):
    @functools.wraps(func)
    def wrapper_decorator(*args, **kwargs):
        # Do something before
        value = func(*args, **kwargs)
        # Do something after
        return value
    return wrapper_decorator
```
## Timing decorator

In [8]:
import functools
import time

def timer(func):
    """Print the runtime of the decorated function"""
    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        start_time = time.perf_counter()    # 1
        value = func(*args, **kwargs) # just a value
        end_time = time.perf_counter()      # 2
        run_time = end_time - start_time    # 3
        print(f"Finished {func.__name__!r} in {run_time:.4f} secs")
        return value
    return wrapper_timer

@timer
def waste_some_time(num_times):
    for _ in range(num_times):
        sum([i**2 for i in range(10000)])
        
waste_some_time(1000)

Finished 'waste_some_time' in 2.1499 secs


# Debugging code
The following @debug decorator will print the arguments a function is called with as well as its return value every time the function is called:

In [11]:
import functools

def debug(func):
    """Print the function signature and return value"""
    @functools.wraps(func)
    def wrapper_debug(*args, **kwargs):
        args_repr = [repr(a) for a in args]                      # 1
        kwargs_repr = [f"{k}={v!r}" for k, v in kwargs.items()]  # 2
        signature = ", ".join(args_repr + kwargs_repr)           # 3
        print(f"Calling {func.__name__}({signature})")
        value = func(*args, **kwargs)
        print(f"{func.__name__!r} returned {value!r}")           # 4
        return value
    return wrapper_debug

The signature is created by joining the string representations of all the arguments. The numbers in the following list correspond to the numbered comments in the code:
* Create a list of the positional arguments. Use repr() to get a nice string representing each argument.
* Create a list of the keyword arguments. The f-string formats each argument as `key=value` where the `!r` specifier means that repr() is used to represent the value.
* The lists of positional and keyword arguments is joined together to one signature string with each argument separated by a comma.
* The return value is printed after the function is executed.

Let’s see how the decorator works in practice by applying it to a simple function with one position and one keyword argument:

In [13]:
@debug
def make_greeting(name, age=None):
    if age is None:
        return f"Howdy {name}!"
    else:
        return f"Whoa {name}! {age} already, you are growing up!"

make_greeting("Benjamin")

make_greeting("Richard", age=112)

make_greeting(name="Dorrisile", age=116)

Calling make_greeting('Benjamin')
'make_greeting' returned 'Howdy Benjamin!'
Calling make_greeting('Richard', age=112)
'make_greeting' returned 'Whoa Richard! 112 already, you are growing up!'
Calling make_greeting(name='Dorrisile', age=116)
'make_greeting' returned 'Whoa Dorrisile! 116 already, you are growing up!'


'Whoa Dorrisile! 116 already, you are growing up!'